In [ ]:
import pandas as pd
import os

cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

pd.options.display.float_format = '{:20,.2f}'.format

establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')
veh_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Vehicle Data')

In [ ]:
tour_data['com_veh_dri_tri'] = tour_data[
    ['company_id', 'vehicle_id', 'driver_id', 'trip_number']
    ].astype(str).agg('-'.join, axis=1)

trip_data['com_veh_dri_tri'] = trip_data[
    ['company_id', 'vehicle_id', 'driver_id', 'trip_number']
    ].astype(str).agg('-'.join, axis=1)

In [ ]:
def custom_lambda(x):
    vals = tour_data[tour_data.com_veh_dri_tri == x]['TRAVEL_DISTANCE'].values
    if len(vals) == 1: return vals[0]
    elif len(vals) == 2:
        if vals[1] != vals[1]:
            if vals[0] == vals[0]: return vals[0]
            else: print(f'trips with conflicting distances: {x}')
        else: print(f'trips with conflicting distances: {x}')
    else: print(f'trips with more than two assigned distances: {x}')

trip_data['trip_dis'] = trip_data.com_veh_dri_tri.apply(
    lambda x: custom_lambda(x)
    )

In [ ]:
a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)+\
    '_'+ tour_data.STOP_SEQ.astype(str)
a.nunique()

In [ ]:
trip_data = trip_data.merge(
    veh_data[['vehicle_classification', 'id']],
    left_on='vehicle_id',
    right_on='id',
    validate='many_to_one',
    how='left'
    )
trip_data = trip_data[trip_data.vehicle_classification != 4]

In [ ]:
trip_data_ = trip_data[trip_data.arrival_time.notna()]


In [ ]:
trip = trip_data_.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
trip = trip[trip.vehicle_classification != 4]
gc = trip[
    ['base_location_Industry Group', 'Most Likely Estimate Weight Factor']
    ].groupby('base_location_Industry Group').sum()
gc

In [ ]:
gc.reset_index()['Most Likely Estimate Weight Factor'].sum()

In [ ]:
len(trip_data_) #matches

In [ ]:
max(tour_id) #matches

In [ ]:
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended, prev = False, False, (None, None)
for se in list(zip(start, end)):
    if se == (True, False):
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    # if ended and not started: counter += 1
    tour_id.append(counter)

In [ ]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

In [ ]:
tour_data['tour_id'] = tour_id

light, med, hvy = list(range(1, 4)), list(range(5, 8)), [4, 8]
def classing(num):
    if num in light: return 'light'
    elif num in med: return 'med'
    else: return 'hvy'
tour_data['VehClass_name'] = tour_data.VehClass.apply(lambda x: classing(x))

In [ ]:
trip_data[trip_data.]

In [ ]:
tour_data_ = tour_data[tour_data.VehClass_name == 'hvy']

tour_data_[['tour_id','Most.Likely.Estimate.Weight.Factor']].groupby('tour_id').max().reset_index()['Most.Likely.Estimate.Weight.Factor'].sum()

In [ ]:
tour_data_[['tour_id','Most.Likely.Estimate.Weight.Factor']].drop_duplicates()['Most.Likely.Estimate.Weight.Factor'].sum()

In [ ]:
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work', 'employees_fulltime_count']]
employees = employees['employees_fulltime_count'].sum()
employees

In [ ]:
2213/46272

In [ ]:
len(trip_data_)/2213

In [ ]:
10687/2213

In [ ]:
trip_data_

In [ ]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE']
].dropna(how='any')
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt_ = vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()
vmt_

In [ ]:
vmt_.weighted_distance.sum()

In [ ]:
# #manual fixing tour_data tour_id
# tour_data.loc[(tour_data.vehicle_id == 117)&
#               (tour_data.TOUR_NUM == 3)&
#               (tour_data.STOP_SEQ == 2), 'tour_id'] = 775
# tour_data.loc[(tour_data.vehicle_id == 176)&
#               (tour_data.TOUR_NUM == 2)&
#               (tour_data.STOP_SEQ == 2)&
#               (tour_data.trip_number == 4),  'tour_id'] = 1530

# a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
# a = {v: k+1 for k, v in enumerate(a.unique())}
# tour_data_ = tour_data[['company_id', 'driver_id', 'vehicle_id','TOUR_NUM', 'trip_number', 'STOP_SEQ', 'TOUR_START', 'TOUR_END']].copy()
# tour_data_['tour_id2'] = tour_id
# tour_data_['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
# tour_data_['tour_id3_'] = tour_data_.tour_id3.apply(lambda x: a[x])
# tour_data_.to_csv('tour_id_verification.csv')

In [ ]:
tour_data.sort_values(['vehicle_id', 'TOUR_NUM','STOP_SEQ'])

In [ ]:
tour_id_ = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended, prev = False, False, (None, None)
for se in list(zip(start, end)):
    if se == (True, False):
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    #if ended and not started: counter += 1
    tour_id_.append(counter)

In [ ]:
tour_id = []
counter = 0
c, d, tp, v, tr = tour_data.company_id, tour_data.driver_id, tour_data.trip_number, tour_data.vehicle_id, tour_data.TOUR_NUM
c_, d_, tp_, v_, tr_ = None, None, None, None, None
for i in range(len(c)):
    if tr[i] != tr_: counter += 1
    else:
        if tp[i] == 0: counter += 1
    c_, d_, tp_, v_, tr_ = c[i], d[i], tp[i], v[i], tr[i]
    tour_id.append(counter)

In [ ]:
a = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
a = {v: k+1 for k, v in enumerate(a.unique())}


In [ ]:
tour_data_ = tour_data[['company_id', 'driver_id', 'vehicle_id','TOUR_NUM', 'trip_number', 'STOP_SEQ', 'TOUR_START', 'TOUR_END']].copy()
tour_data_['tour_id1'] = tour_id_
tour_data_['tour_id2'] = tour_id
tour_data_['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
tour_data_['tour_id3_']= tour_data_.tour_id3.apply(lambda x: a[x])
tour_data_

In [ ]:
tour_data_.to_csv('tour_id_verification.csv')

In [ ]:
max(tour_id_)

In [ ]:
max(a.values())

In [ ]:
tour_data['tour_id3'] = tour_data.vehicle_id.astype(str)+'_'+tour_data.TOUR_NUM.astype(str)
tour_data['tour_id1'] = tour_id_
tour_data['tour_id2'] = tour_id

In [ ]:
tour_1 = tour_data[['Most.Likely.Estimate.Weight.Factor', 'tour_id1']].groupby('tour_id1').mean().reset_index()
tour_1 = tour_1.merge(
    tour_data[['base_location_Industry Group', 'tour_id1']].groupby('tour_id1').mean(),
    on='tour_id1',
    how='left',
    validate='one_to_many'
    )
tour_1[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group']
].groupby('base_location_Industry Group').sum().reset_index()['Most.Likely.Estimate.Weight.Factor'].sum()

In [ ]:
tour_data.used_other_vehicle.value_counts()

In [ ]:
tour_data.sort_values(['vehicle_id', 'TOUR_NUM','STOP_SEQ'])